In [2]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

1:""
2:"Team"
3:"2020/21"
4:"2020/21(*)"


In [2]:
# webscraping function used to grab team salary data from 1990 - 2021 from https://hoopshype.com/salaries/2019-2020/
# The data presented on this website was from the HoopsHype's salary database and the adjusted for inflation data points
# were provided by Current US Dollars adjusted for inflation from data provided by the U.S.
# Department of Labor Bureau of Labor Statistic
# The function is based of the code from Syed Sadat Nazrul written for a tutorial on Web Scraping with python
# written on Jul 25,2018 https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
def salary(url,year):
    url=url
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    col = []
    i=0

    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        print ('%d:"%s"'%(i,name))
        col.append((name,[]))
    for j in range(1,len(tr_elements)):
        T=tr_elements[j]
        if len(T)!=4:
            break
        i=0
        for t in T.iterchildren():
            data=t.text_content()
            #check if row is empty
            if i>0:
                try:
                    data=int(data)
                except:
                    pass
            col[i][1].append(data)
            i+=1

    [len(c) for (title,c) in col]

    dict = {title:column for (title,column)in col}
    df=pd.DataFrame(dict)

    team = df['Team'].str.strip('\n\t')
    rank = df[''].str.strip('\n\t.')
    salary = df.iloc[:,2].str.strip('\n\t$')
    salary2 = df.iloc[:,3].str.strip('\n\t$')
    year = [year]*27

    teamSalary = pd.DataFrame({
        "rank":rank[0:27],
        "team":team[0:27],
        "salary": salary[0:27],
        "salary(Inflation)":salary2[0:27],
        "Year":year
    })
    return teamSalary

df = salary("https://hoopshype.com/salaries/2020-2021/",2021)
df.to_csv(r'D:\UofC2021\DATA601\Project\Data601_Project\salary21.csv',index = False)

NameError: name 'requests' is not defined